In [40]:
"""
Created on Wed May 14 13:45:38 2021
@author: 胡琳苑
"""

import sys
import math
import time
import random
import gurobipy as gb
from gurobipy import GRB
from gurobipy import *

In [41]:
# instanceGen
'''
w:the number of worktables
s:the number of total SKU
R:the number of ranks
r:the number of skus in one rank,please enter a list,r >=  SKUs in ORDER/R
first element is lb and the second element is ub,and [lb,ub]
O:the number of orders
o:the number of skus in one Order,same as 'r'
'''
def instanceGEN(w,s,sn,R,r,rn,O,o):
    P   = range(w)
    SKU = range(s)
    used_sku =[]
    Order= dict()
    Order_dict = dict()
    for i in range(O):
            J = random.randint(o[0],o[1])
            sku_list =[]
            while len(sku_list)<J:
                rsku = random.choice(SKU)
                used_sku.append(rsku)
                sku_list.append(int(rsku))
                sku_list =list(set(sku_list))
            sku_dict = dict()
            skudict = dict()
            for s in sku_list:
                sku_dict.update({s:random.randint(sn[0],sn[1])})
                skudict.update({s:0})
                
            Order.update({i:sku_dict})
            Order_dict.update({i:skudict})
            
    avail_sku = list(set(used_sku))   
    
    # 从订单中使用的sku生成货架
        
    Rank =dict()
    ri = 0
    for i in range(R):
        J = random.randint(r[0],r[1])
        rank_list =[]
        
        while len(rank_list)<J:
            if ri < len(avail_sku):
                rank_list.append(avail_sku[ri])
                ri = ri + 1
            rank_list.append(random.choice(SKU))
            rank_list =list(set(rank_list))
            
        rack_dict = dict()
        rackdict = dict()
        for rr in rank_list:
            rack_dict.update({rr:random.randint(rn[0],rn[1])})
            rackdict.update({rr:0})
            
        Rank.update({i:rack_dict})

    Ri = dict()
    for si in range(s):
        ranksetfors = []
        for r in range(R):
            if SKU[si] in Rank[r]:
                ranksetfors.append(r)           
        Ri.update({si: set(ranksetfors)})
        
    return P,SKU,Order,Order_dict,Rank,Ri

def solveORAP(P,SKU,Order,Rank,Ri,C,Q,opf):
    Model = gb.Model('OARAP')
    Model.setParam('OutputFlag', opf)

    # desicion variable
    x_op = dict()
    for o in Order:
        for p in P:
             x_op.update({(o,p):Model.addVar(vtype=GRB.BINARY,name=f'x_{o},{p}')})

    u_r = dict()
    for r in Rack:
            u_r.update({r:Model.addVar(vtype=GRB.BINARY,name=f'u_{r}')})

    y_rp= dict()
    for r in Rack:
        for p in P:
            y_rp.update({(r,p):Model.addVar(vtype=GRB.BINARY,name=f'y_{r}{p}')})

    # 货架r给工作台p多少个货i
    pi_irp = dict()
    for p in P:
        for r in Rack:
            for i in SKU:
                pi_irp.update({(i,r,p):Model.addVar(vtype=GRB.INTEGER,name=f'pi_{i}{r}{p}')})

    # 货架 r 有的货i 的个数
    s_ir = dict()
    for r in Rack:        
        for i in SKU:
            if i in Rack[r]:
                s_ir.update({(i,r):Rack[r][i]})
            else:
                s_ir.update({(i,r):0})


    # 订单 o 有的货 i的个数
    q_io = dict()
    for o in Order:
        for i in SKU:
            if i in Order[o]:
                q_io.update({(i,o):Order[o][i]})
            else:
                q_io.update({(i,o):0})

    # 目标函数 (18)
    Model.setObjective(gb.quicksum(u_r[r] for r in Rack),GRB.MINIMIZE)      

    # (19)
    for p in P:
        Model.addConstr(gb.quicksum(x_op[o,p] for o in Order) == Q[p])

    #(20)
    for o in Order:
        Model.addConstr(gb.quicksum(x_op[o,p] for p in P) <= 1)

    #(21)   
    for p in P:
        for i in SKU:
            q_sum = 0
            for o in Order:
                    q_sum =q_sum + q_io[i,o]
            if q_sum >=1:
                Model.addConstr(gb.quicksum(pi_irp[i,r,p] for r in Rack) >=
                        gb.quicksum(q_io[i,o]*x_op[o,p] for o in Order))
    #(22)
    for i in SKU:
        for r in Rack:
            for p in P:
                Model.addConstr(pi_irp[i,r,p]<=s_ir[i,r]*y_rp[r,p])


    #(23)
    for i in SKU:
        for r in Rack:
            Model.addConstr(quicksum(pi_irp[i,r,p] for p in P)<=s_ir[i,r])

    #(24)
    for r in Rack:
        for p in P:
            Model.addConstr(y_rp[r,p]<=u_r[r])

    Model.optimize()
    if Model.status == GRB.Status.INFEASIBLE:
        print('Optimization was stopped with status %d' % Model.status)
    # do IIS, find infeasible constraints
        Model.computeIIS()
        for c in Model.getConstrs():
            if c.IISConstr:
                print('%s' % c.constrName)
        return 0
    else:
        U = Model.getAttr('x',u_r)
        X = Model.getAttr('x',x_op)
        Y = Model.getAttr('x',y_rp)
        PI = Model.getAttr('x',pi_irp)
            
    return U,X,Y,PI

In [63]:
P,SKU,Order,Order_dict,Rack,Ri = instanceGEN(w=30,s=40,R=40,r=[4,6],rn=[35,40],O=200,o=[2,4],sn=[1,3])

In [64]:
# OAPA-matheuristic 
m = len(P)     # 工作台数量
n = len(Order) # 待处理订单数量
R = len(Rack)  # 货架的数量
S = len(SKU)   # SKU的数量
C = 5          # 工作台容量
T = 30         # 处理期间
Q = []         # 订单分配
opf = 1
for p in range(m):
    trans_p = p+1
    if (trans_p>=1) & (trans_p<=n%m):
            Q.append(math.ceil(n/m))
    else:
        Q.append(math.floor(n/m))

order = dict()
for o in Order:
    order.update({o:Order[o]})

timeused = 0 
for p in P:
   
    start = time.time()
    U,X,Y,PI = solveORAP([p],SKU,order,Rack,Ri,C,Q,0)
    end = time.time()
    timeused = timeused + (end-start)
    # 留下未分配的订单
    print(timeused)
    orderleft = dict()
    for o in order:            
        if X[o,p] == 0:
            orderleft.update({o:Order[o]})
    order = orderleft
    
    rackused = dict()
    rack = dict()
    
    for r in Rack:
        if Y[r,p] !=0: 
            rackused.update({r:Rack[r]}) 
    
    
Ri = dict()
for si in SKU:
    ranksetfors = []
    for r in rackused:
        if SKU[si] in rackused[r]:
            ranksetfors.append(r)           
    Ri.update({si: set(ranksetfors)})

start = time.time()
Ue,Xe,Ye,PIe = solveORAP(P,SKU,Order,rackused,Ri,C,Q,1)   
end = time.time()
timeused = timeused + (end-start)
print(timeused)    

0.24805045127868652
0.4430973529815674
0.6412453651428223
0.8280565738677979
0.9966659545898438
1.2021801471710205
1.3888967037200928
1.795595407485962
2.114508867263794
2.463704824447632
2.7343881130218506
3.0794897079467773
3.4329586029052734
3.752622127532959
4.063135862350464
4.631881237030029
5.231845855712891
5.473006010055542
6.014747619628906
6.539090156555176
6.9883668422698975
7.406041860580444
7.649696588516235
7.807301998138428
8.119070291519165
8.263099908828735
8.38271427154541
8.500556945800781
8.58393383026123
8.63837718963623
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 52230 rows, 55240 columns and 182220 nonzeros
Model fingerprint: 0x9ca53815
Variable types: 0 continuous, 55240 integer (7240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
 

In [65]:
# gurobi only 
m = len(P)     # 工作台数量
n = len(Order) # 待处理订单数量
R = len(Rack)  # 货架的数量
S = len(SKU)   # SKU的数量
C = 5          # 工作台容量
Q = []         # 订单分配
opf = 1
for p in range(m):
    trans_p = p+1
    if (trans_p>=1) & (trans_p<=n%m):
            Q.append(math.ceil(n/m))
    else:
        Q.append(math.floor(n/m))
timeused = 0
start = time.time()
Ug,Xg,Yg,PIg = solveORAP(P,SKU,Order,Rack,Ri,C,Q,1)
end = time.time()
timeused = timeused + (end-start)
print(timeused)  

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 52230 rows, 55240 columns and 182220 nonzeros
Model fingerprint: 0x9ca53815
Variable types: 0 continuous, 55240 integer (7240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 45709 rows and 43170 columns
Presolve time: 0.15s
Presolved: 6521 rows, 12070 columns, 51668 nonzeros
Variable types: 0 continuous, 12070 integer (6040 binary)

Root relaxation: objective 3.451000e+00, 18139 iterations, 3.11 seconds
Total elapsed time = 6.53s
Total elapsed time = 12.87s
Total elapsed time = 15.54s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

In [33]:
U,X,Y,PI = solveORAP([0],SKU,order,Rack,Ri,C,Q,0)

In [35]:
PI[8,1,0]

21.0

In [1]:
# gurobi only 
# 下标
m = len(P)     # 工作台数量
n = len(Order) # 待处理订单数量
R = len(Rank)  # 货架的数量
S = len(SKU)   # SKU的数量
C = 5          # 工作台容量
T = 30         # 处理期间
Q = []         # 订单分配
opf = 1
for p in range(m):
    trans_p = p+1
    if (trans_p>=1) & (trans_p<=n%m):
            Q.append(math.ceil(n/m))
    else:
        Q.append(math.floor(n/m))
        
Model = gb.Model('OARAP')
Model.setParam('OutputFlag', opf)

# desicion variable
x_op = dict()
for o in Order:
    for p in P:
         x_op.update({(o,p):Model.addVar(vtype=GRB.BINARY,name=f'x_{o},{p}')})

u_r = dict()
for r in Rack:
        u_r.update({r:Model.addVar(vtype=GRB.BINARY,name=f'u_{r}')})

y_rp= dict()
for r in Rack:
    for p in P:
        y_rp.update({(r,p):Model.addVar(vtype=GRB.BINARY,name=f'y_{r}{p}')})

# 货架r给工作台p多少个货i
pi_irp = dict()
for p in P:
    for r in Rack:
        for i in SKU:
            pi_irp.update({(i,r,p):Model.addVar(vtype=GRB.INTEGER,name=f'pi_{i}{r}{p}')})

# 货架 r 有的货i 的个数
s_ir = dict()
for r in Rack:        
    for i in SKU:
        if i in Rack[r]:
            s_ir.update({(i,r):Rack[r][i]})
        else:
            s_ir.update({(i,r):0})
            

# 订单 o 有的货 i的个数
q_io = dict()
for o in Order:
    for i in SKU:
        if i in Order[o]:
            q_io.update({(i,o):Order[o][i]})
        else:
            q_io.update({(i,o):0})

# 目标函数 (18)
Model.setObjective(gb.quicksum(u_r[r] for r in Rack),GRB.MINIMIZE)      

# (19)
for p in P:
    Model.addConstr(gb.quicksum(x_op[o,p] for o in Order) == Q[p])
    
#(20)
for o in Order:
    Model.addConstr(gb.quicksum(x_op[o,p] for p in P) <= 1)

#(21)   
for p in P:
    for i in SKU:
        q_sum = 0
        for o in Order:
                q_sum =q_sum + q_io[i,o]
        if q_sum >=1:
            Model.addConstr(gb.quicksum(pi_irp[i,r,p] for r in Rack) >=
                    gb.quicksum(q_io[i,o]*x_op[o,p] for o in Order))
#(22)
for i in SKU:
    for r in Rack:
        for p in P:
            Model.addConstr(pi_irp[i,r,p]<=s_ir[i,r]*y_rp[r,p])

            
#(23)
for i in SKU:
    for r in Rack:
        Model.addConstr(quicksum(pi_irp[i,r,p] for p in P)<=s_ir[i,r])
        
#(24)
for r in Rack:
    for p in P:
        Model.addConstr(y_rp[r,p]<=u_r[r])

Model.optimize()
if Model.status == GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % Model.status)
    # do IIS, find infeasible constraints
    Model.computeIIS()
    for c in Model.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)

NameError: name 'P' is not defined